# <a id='toc1_'></a>[Guardrails Strategies](#toc0_)

In this notebook, we showcase different guardrails strategies that can be used separately or combined to ensure the generated answers are not hallucinated or harmfull. We will: 
- Ensure the generated answer is grounded in the context retrieved, minimizing hallucinations using: 
  - Custom methods
  - Ragas Framework
- See other gardrails strategies using Giskard scanner

**Table of contents**<a id='toc0_'></a>    
- [Set Up](#toc2_)    
- [1) Custom method: Verify that the answer is grounded in the context retrieved](#toc3_)    
- [2) Ragas framework: Faithfullness & other metrics](#toc4_)    
- [3) Giskard: a testing framework for LLM applications.](#toc5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc2_'></a>[Set Up](#toc0_)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

os.chdir(Path.cwd().joinpath(".."))
print(Path.cwd())
load_dotenv(override=True)

In [ ]:
from operator import itemgetter

from datasets import Dataset
from langchain.prompts import ChatPromptTemplate
from langchain.schema import Document, StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.runnables import RunnableLambda, RunnableParallel
from ragas import evaluate
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import (
    Faithfulness,
    ResponseRelevancy,
)

from lib.models import embeddings, llm
from lib.utils import (
    build_vector_store,
    load_documents,
    load_vector_store,
    split_documents_basic,
)

In [ ]:
BASE_CHUNK_SIZE = 1024

# build vector_store
base_documents = split_documents_basic(load_documents("data/5_docs"), BASE_CHUNK_SIZE, include_linear_index=True)

build_vector_store(
    base_documents,
    embeddings,
    collection_name="5_docs",
    distance_function="cosine",
    erase_existing=False,
)

# Load Vector store / retriever
chroma_vector_store = load_vector_store(embeddings, "5_docs")
chroma_vector_store_retriever = chroma_vector_store.as_retriever()

In [ ]:
questions = [
    "According to the IPCC report, what are key risks in the Europe?",
    "Is sea level rise avoidable?",
    "Will sea level rise stop?",
    "What are the main climate risks in North America?",
]

To illustrate the use of the 2 guardrail strategies showcased in this notebook, we will create a simple Retrieval-Augmented Generation (RAG) pipeline.

In [ ]:
PROMPT_TEMPLATE = """You are the Climate Assistant, a helpful AI assistant.
Your task is to answer common questions on climate change.
You will be given a question and relevant excerpts from the IPCC Climate Change Synthesis Report (2023).
Please provide short and clear answers based on the provided context. Be polite and helpful.

Context:
{context}

Question:
{question}

Your answer:
"""


def _format_docs(docs: list[Document]) -> list[str]:  # noqa: UP006
    return [doc.page_content for doc in docs]


def _concate_chunk(chunks: list[str]) -> str:  # noqa: UP006
    return "\n\n".join(chunk for chunk in chunks)


response_prompt = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)

rag_chain = (
    RunnableParallel(
        {
            "source_documents": chroma_vector_store_retriever,
            "question": RunnablePassthrough(),
        }
    )
    .assign(chunks=RunnableLambda(itemgetter("source_documents")) | _format_docs)
    .assign(context=RunnableLambda(itemgetter("chunks")) | _concate_chunk)
    .assign(answer=response_prompt | llm | StrOutputParser())
)

In [ ]:
responses = rag_chain.batch(questions)
print(responses[0]["answer"])

# <a id='toc3_'></a>[1) Custom method: Verify that the answer is grounded in the context retrieved](#toc0_)

In this section, we implemented the function `check_grounded_in_context` that determines if the response is grounded in the retrieved context. The function returns 'GROUNDED' if the response is fully supported by the context, and 'NOT_GROUNDED' otherwise.

In [ ]:
# Ensure response is grounded in context
def check_grounded_in_context(response: str, context: list[Document]) -> bool:
    prompt = ChatPromptTemplate.from_template(
        """Given the following context and response, determine if the response is fully grounded in the context. \
If it is, return 'GROUNDED'. If it contains information not present in the context, return 'NOT_GROUNDED'.\n\n
Context: {context}\n
Response: {response}\n
Determination:"""
    )
    chain = prompt | llm | StrOutputParser()
    result = chain.invoke({"context": context, "response": response})
    return result.strip() == "GROUNDED"

In [ ]:
for indx, response in enumerate(responses):
    is_grounded = check_grounded_in_context(response["answer"], response["context"])
    print(f"question #{indx} : {is_grounded}")

# <a id='toc4_'></a>[2) Ragas framework: Faithfullness & other metrics](#toc0_)

Ragas is an open-source evaluation framework for Retrieval-Augmented Generation (RAG) systems, providing standardized metrics to measure answer quality, relevance, and faithfulness. It helps developers benchmark and improve RAG pipelines with reproducible and comparable results. 

We will use 2 metrics:

1. [`Faithfulness`](https://docs.ragas.io/en/stable/concepts/metrics/available_metrics/faithfulness/#faithfulness) - Measures the factual consistency of the answer to the context based on the question. A low score indicates that the answer contains hallucinations (information not supported by the context), while a high score means the response is faithful to the source.
2. [`ResponseRelevancy`](https://docs.ragas.io/en/stable/concepts/metrics/available_metrics/answer_relevance/#faithfullness-with-hhem-21-open) - Measures how relevant the answer is to the question.
the question.

To explore other metrics, check the [metrics guide](https://docs.ragas.io/en/stable/concepts/metrics/).

In [ ]:
# Wrap llm & embeddings for Ragas
ragas_llm = LangchainLLMWrapper(llm)
ragas_embeddings = LangchainEmbeddingsWrapper(embeddings)

# Build dataset for Ragas
data = {
    "question": questions,
    "answer": [out["answer"] for out in responses],
    "contexts": [out["chunks"] for out in responses],
}

dataset = Dataset.from_dict(data)

metrics = [
    Faithfulness(llm=ragas_llm),
    ResponseRelevancy(llm=ragas_llm, embeddings=ragas_embeddings),
]

In [ ]:
dataset

In [ ]:
ragas_results = evaluate(dataset=dataset, metrics=metrics, llm=ragas_llm, embeddings=ragas_embeddings)

In [ ]:
ragas_results.to_pandas()

# <a id='toc5_'></a>[3) Giskard: a testing framework for LLM applications.](#toc0_)

Giskard is an open-source framework that helps data scientists and machine learning teams automatically test, debug, and monitor their models for biases, errors, and vulnerabilities. It enables safe and reliable deployment of AI systems by providing collaborative tools to evaluate model quality and trustworthiness.

Giskard is a broad AI testing framework for detecting biases and vulnerabilities in any ML model, while Ragas focuses specifically on evaluating RAG pipelines for LLMs.


In [ ]:
import giskard
import pandas as pd

giskard.llm.set_llm_model(
    f"azure/{os.getenv('LLM_AZURE_OPENAI_DEPLOYMENT_NAME')}",
    api_base=os.getenv("LLM_AZURE_OPENAI_ENDPOINT"),
    api_version=os.getenv("LLM_AZURE_OPENAI_API_VERSION"),
    api_key=os.getenv("LLM_AZURE_OPENAI_API_KEY"),
)

giskard.llm.set_embedding_model(
    f"azure/{os.getenv('LLM_AZURE_OPENAI_DEPLOYMENT_NAME')}",
    api_base=os.getenv("EMBEDDINGS_AZURE_OPENAI_ENDPOINT"),
    api_version=os.getenv("EMBEDDINGS_AZURE_OPENAI_API_VERSION"),
    api_key=os.getenv("EMBEDDINGS_AZURE_OPENAI_API_KEY"),
)


def model_predict(df: pd.DataFrame) -> list:
    """Wraps the LLM call in a simple Python function.

    The function takes a pandas.DataFrame containing the input variables needed
    by your model, and must return a list of the outputs (one for each row).
    """
    return [rag_chain.invoke(question)["answer"] for question in df["question"]]


# Don’t forget to fill the `name` and `description`: they are used by Giskard
# to generate domain-specific tests.
giskard_model = giskard.Model(
    model=model_predict,
    model_type="text_generation",
    name="Climate Change Question Answering",
    description="This model answers any question about climate change based on IPCC reports",
    feature_names=["question"],
)

In [ ]:
giskard_dataset = giskard.Dataset(pd.DataFrame({"question": questions}), target=None)

print(giskard_model.predict(giskard_dataset).prediction)

In [ ]:
# for hallucination
report = giskard.scan(giskard_model, giskard_dataset, only="hallucination")

# for the full report, run (can take several minutes)
# report = giskard.scan(giskard_model, giskard_dataset)

In [ ]:
display(report)

Note: The hallucination check from the Ragas is more a Sycophancy Detector. See [Giskard's documentation](https://docs.giskard.ai/en/stable/knowledge/llm_vulnerabilities/index.html#llm-assisted-detectors) for more details. 

> Sycophancy detector
> The sycophancy detector (see :class:~giskard.scanner.llm.LLMBasicSycophancyDetector) is an example of an LLM-assisted detector. Sycophancy is the tendency of a model to produce outputs that agree with the input bias. This is often linked to model hallucination, and allows us to test for model coherency and hallucination even when we don’t have access to specific ground truth data to verify the model outputs.  
> To detect sycophantic behavior, we will use an LLM to generate pairs of adversarial inputs tailored for the model under test. Each pair will contain queries that are biased in opposite ways, but which should produce the same answer from the model.  
> As an example, consider a question-answering model on climate change based on reporting by the IPCC (Intergovernmental Panel on Climate Change). Our LLM-assisted input generation will generate pairs of questions, at least one of which will have a specific bias or make assumptions that contradict the other."


When we use the Giscard scan, we only use the question and the output generated and an other LLM is the judge.
It's also possible to have custom metric or to embedd ragas metrics in Giskard.

In [ ]:
report.to_html("output/5_giskard_report.html")